In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pathlib
import numpy as np

# Aumenta a qualidade das imagens plotadas no notebook
plt.rcParams["figure.dpi"] = 100

In [ ]:
# Caminho para os dataset
data_dir = pathlib.Path('dataset/')

# Define os caminhos para treino e validação
train_dir = data_dir / 'training'
validation_dir = data_dir / 'validation'
evaluation_dir = data_dir / 'evaluation'

# Parâmetros do modelo
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32 # Quantas imagens o modelo vê por vez

In [6]:
# Carrega o dataset de treino
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  labels='inferred',
  label_mode='int', # Rótulos como inteiros (0 para Bread, 1 para Dairy, etc.)
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

# Carrega o dataset de validação
val_ds = tf.keras.utils.image_dataset_from_directory(
  validation_dir,
  labels='inferred',
  label_mode='int',
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

# Guarda os nomes das classes para usar depois
class_names = train_ds.class_names
print("Classes encontradas:", class_names)

Found 9866 files belonging to 11 classes.
Found 3430 files belonging to 11 classes.
Classes encontradas: ['Bread', 'Dairy product', 'Dessert', 'Egg', 'Fried food', 'Meat', 'Noodles-Pasta', 'Rice', 'Seafood', 'Soup', 'Vegetable-Fruit']


In [7]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Camada de Data Augmentation
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
  ]
)

# Modelo Base
# A camada de reescala já está incluída no EfficientNet
base_model = tf.keras.applications.EfficientNetV2B0(
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    include_top=False, # Não incluir a camada classificadora final
    weights='imagenet' # Carregar pesos pré-treinados na ImageNet
)

# Congelar o modelo base para não retreiná-lo inicialmente
base_model.trainable = False

# Montagem do Modelo Completo
inputs = keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
x = data_augmentation(inputs)
x = base_model(x, training=False) # 'training=False' é importante aqui
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x) # Dropout para regularização
outputs = layers.Dense(len(class_names))(x) # Camada final com 11 neurônios

model = keras.Model(inputs, outputs)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-b0 (Functional)  │ (None, 7, 7, 1280)     │     5,919,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 11)             │        14,091 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,933,403 (22.63 MB)

 Trainable params: 14,091 (55.04 KB)

 Non-trainable params: 5,919,312 (22.58 MB)

In [9]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [10]:
EPOCHS = 20

history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=EPOCHS
)

Epoch 1/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 109s 331ms/step - accuracy: 0.5869 - loss: 1.3166 - val_accuracy: 0.8292 - val_loss: 0.5537
Epoch 2/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 87s 282ms/step - accuracy: 0.8125 - loss: 0.5774 - val_accuracy: 0.8551 - val_loss: 0.4612
Epoch 3/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 86s 279ms/step - accuracy: 0.8420 - loss: 0.4903 - val_accuracy: 0.8685 - val_loss: 0.4256
Epoch 4/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 87s 282ms/step - accuracy: 0.8514 - loss: 0.4599 - val_accuracy: 0.8723 - val_loss: 0.4057
Epoch 5/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 88s 286ms/step - accuracy: 0.8594 - loss: 0.4319 - val_accuracy: 0.8787 - val_loss: 0.3870
Epoch 6/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 88s 285ms/step - accuracy: 0.8642 - loss: 0.4179 - val_accuracy: 0.8805 - val_loss: 0.3778
Epoch 7/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 87s 281ms/step - accuracy: 0.8678 - loss: 0.3970 - val_accuracy: 0.8854 - val_loss: 0.3680
Epoch 8/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 88s 285ms/step - accuracy: 0.8774 - loss: 

In [11]:
# Define o caminho e o nome do arquivo para salvar o modelo
model_save_path = 'food11_efficientnet_model.keras'

# Salva o modelo inteiro (arquitetura, pesos e configuração do otimizador)
model.save(model_save_path)

print(f"Modelo salvo com sucesso em: {model_save_path}")

Modelo salvo com sucesso em: food11_efficientnet_model.keras


In [1]:
# Função para carregar uma imagem e fazer a previsão
def predict_image(image_path):
    img = keras.utils.load_img(
        image_path, target_size=(IMG_HEIGHT, IMG_WIDTH)
    )
    img_array = keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Cria um batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0]) # Aplica softmax para obter probabilidades

    predicted_class = class_names[np.argmax(score)]
    confidence = 100 * np.max(score)

    plt.imshow(img)
    plt.title(f"Predição: {predicted_class}\nConfiança: {confidence:.2f}%")
    plt.axis("off")
    plt.show()


# Teste com uma imagem da pasta de avaliação
# Mude o nome do arquivo para testar outras imagens
predict_image(evaluation_dir / 'Meat/123.jpg')

NameError: name 'evaluation_dir' is not defined